In [1]:
from dotenv import load_dotenv
import os
import requests
import json
from typing import Optional, Dict, List

load_dotenv()

True

In [ ]:
class AgmarknetAPI:
    """
    Client for accessing Indian agricultural market data via data.gov.in API
    """
    
    def __init__(self):
        """
        Initialize API client
        
        Args:
            api_key: API key from data.gov.in (default is public key with 10 record limit)
        """
        
        # self.base_url = "https://api.data.gov.in/resource/9ef84268-d588-465a-a308-a864a43d0070"
        self.base_url = "https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24"
        self.api_key = os.getenv("DATA_GOV_IN_API_KEY")
    
    def get_commodity_prices(
        self,
        commodity: Optional[str] = None,
        state: Optional[str] = None,
        district: Optional[str] = None,
        market: Optional[str] = None,
        variety: Optional[str] = None,
        grade: Optional[str] = None,
        limit: int = 10,
        offset: int = 0
    ) -> Dict:
        """
        Fetch commodity prices from mandis
        
        Args:
            commodity: Name of commodity (e.g., "Tomato", "Potato", "Onion")
            state: State name (e.g., "Karnataka", "Maharashtra")
            district: District name (e.g., "Bangalore")
            market: Market name (e.g., "Bangalore")
            limit: Number of records to fetch (max 10 for public API key)
            offset: Offset for pagination
            
        Returns:
            Dictionary containing API response with price data
        """

        params = {
            "api-key": self.api_key,
            "format": "json",
            "limit": limit,
            "offset": offset
        }
        
        # Add filters if provided
        if commodity:
            params["filters[commodity]"] = commodity
        if state:
            params["filters[state]"] = state
        if district:
            params["filters[district]"] = district
        if market:
            params["filters[market]"] = market
        if variety:
            params["filters[variety]"] = variety
        if grade:
            params["filters[grade]"] = grade
            
        try:
            response = requests.get(self.base_url, params=params, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            return {"error": str(e), "status": "failed"}
        
    
    def search_by_commodity(self, commodity: str, limit: int = 10) -> Dict:
        """
        Search prices for a specific commodity across all markets
        
        Args:
            commodity: Commodity name (e.g., "Tomato")
            limit: Number of records
            
        Returns:
            Price data for the commodity
        """
        return self.get_commodity_prices(commodity=commodity, limit=limit)
    
    def search_by_location(self, state: str, limit: int = 10) -> Dict:
        """
        Get all commodity prices for a specific state
        
        Args:
            state: State name
            limit: Number of records
            
        Returns:
            Price data for the state
        """
        return self.get_commodity_prices(state=state, limit=limit)
    
    def search_specific(
        self,
        commodity: str,
        state: str,
        market: Optional[str] = None
    ) -> Dict:
        """
        Get price for specific commodity in specific location
        
        Args:
            commodity: Commodity name
            state: State name
            market: Market name (optional)
            
        Returns:
            Specific price data
        """
        return self.get_commodity_prices(
            commodity=commodity,
            state=state,
            market=market,
            limit=10
        )
    
    def pretty_print(self, data: Dict) -> None:
        """
        Pretty print API response
        
        Args:
            data: API response dictionary
        """
        if "error" in data:
            print(f"Error: {data['error']}")
            return
            
        print(json.dumps(data, indent=2, ensure_ascii=False))

    def parse_records(self, data: Dict) -> List[Dict]:
        """
        Extract and parse records from API response
        
        Args:
            data: API response dictionary
            
        Returns:
            List of parsed records
        """
        if "error" in data:
            return []
        
        # The API response structure might vary, adjust based on actual response
        if "records" in data:
            return data["records"]
        return []

In [3]:
# Search for tomato prices across india
api = AgmarknetAPI()
data = api.search_by_commodity("tomato", limit=20)
api.pretty_print(data)

{
  "created": 1627939168,
  "updated": 1760814080,
  "created_date": "2021-08-02T21:19:28Z",
  "updated_date": "2025-10-18T19:01:20Z",
  "active": "1",
  "index_name": "9ef84268-d588-465a-a308-a864a43d0070",
  "org": [
    "Ministry of Agriculture and Farmers Welfare",
    "Department of Agriculture and Farmers Welfare"
  ],
  "org_type": "Central",
  "source": "data.gov.in",
  "title": "Current Daily Price of Various Commodities from Various Markets (Mandi)",
  "external_ws_url": "",
  "visualizable": "1",
  "field": [],
  "external_ws": 0,
  "catalog_uuid": "6141ea17-a69d-4713-b600-0a43c8fd9a6c",
  "sector": [
    "Agriculture",
    "Agricultural Marketing"
  ],
  "target_bucket": {
    "field": "9ef84268-d588-465a-a308-a864a43d0070",
    "index": "daily_mandi_price",
    "type": "6141ea17-a69d-4713-b600-0a43c8fd9a6c"
  },
  "desc": "Current Daily Price of Various Commodities from Various Markets (Mandi)",
  "field_exposed": [
    {
      "name": "State",
      "id": "state.keyword"

In [ ]:
# Search for tomato prices across india
api = AgmarknetAPI()
data = api.search_by_commodity("Tomato", limit=20)
api.pretty_print(data)

In [ ]:
# search all commodities in karnataka
data = api.search_by_location("Uttar Pradesh")
api.pretty_print(data)